In [2]:
import torch
from torch import nn
from d2l import torch as d2l
import pandas as pd

In [3]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [4]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [10]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 3
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 1000))

VGGNet = vgg(conv_arch)

In [11]:
X = torch.randn(size=(1, 3, 224, 224))
for blk in VGGNet:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 1000])


In [12]:
num_sum = 0
rec = []
for l in VGGNet:
    if isinstance(l, nn.Sequential):
        for l2 in l:
            if isinstance(l2, nn.Conv2d):
                num = l2.in_channels * l2.out_channels * l2.kernel_size[0] * l2.kernel_size[1]
                num_sum += num
                rec.append([l2.__class__.__name__, num])
    if isinstance(l, nn.Linear):
        num = l.in_features * l.out_features
        num_sum += num
        rec.append([l.__class__.__name__, num])
        
print(f'{num_sum} = {round(num_sum / 1000 / 1000, 1)}M')
pd.DataFrame(rec, columns=['name', 'parameter nums'])

132851392 = 132.9M


,name,parameter nums
0,Conv2d,1728
1,Conv2d,73728
2,Conv2d,294912
3,Conv2d,589824
4,Conv2d,1179648
5,Conv2d,2359296
6,Conv2d,2359296
7,Conv2d,2359296
8,Linear,102760448
9,Linear,16777216
